In [1]:
import pandas as pd
import numpy as np
# import os
# os.chdir('/Users/weiqinggao/Downloads')
import nltk
nltk.data.path.append(r"C:\Users\liufa\nltk_data")

# from nltk.tokenize import word_tokenize

# nltk.download('brown')
import langdetect
import matplotlib.pyplot as plt
import collections
from sklearn.model_selection import train_test_split

In [2]:
raw_data=pd.read_csv(r'C:\Users\liufa\Desktop\Kaggle competition\train.csv')

df = raw_data.rename(columns={'excerpt':'text','target':'y'})
df.head(5)

,id,url_legal,license,text,y,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [3]:
#language detection
df['lang'] = df['text'].apply(lambda x: langdetect.detect(x))
df['lang'].value_counts()

en    2834
Name: lang, dtype: int64

In [4]:
import string
string.punctuation

def remove_punct(text):
    text_nopunct=''.join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['text_no_punct']=df['text'].apply(lambda x: remove_punct(x))
df.head()

,id,url_legal,license,text,y,standard_error,lang,text_no_punct
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,en,When the young people returned to the ballroom...
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,en,All through dinner time Mrs Fayre was somewhat...
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,en,As Roger had predicted the snow departed as qu...
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,en,And outside before the palace a great garden w...
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,en,Once upon a time there were Three Bears who li...


In [5]:
import re
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['text_tokenized']=df['text_no_punct'].apply(lambda x: tokenize(x.lower()))
df.head()

,id,url_legal,license,text,y,standard_error,lang,text_no_punct,text_tokenized
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,en,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ..."
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,en,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ..."
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,en,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe..."
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,en,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ..."
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,en,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear..."


In [6]:
import nltk
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(tokenized_list):
    text=[word for word in tokenized_list if word not in stopwords]
    return text
df['text_no_stopwords']=df['text_tokenized'].apply(lambda x: remove_stopwords(x))
df.head()

,id,url_legal,license,text,y,standard_error,lang,text_no_punct,text_tokenized,text_no_stopwords
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,en,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,..."
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,en,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e..."
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,en,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca..."
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,en,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round..."
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,en,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho..."


In [7]:
wn=nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text=[wn.lemmatize(word) for word in tokenized_text]
    return text
df['text_lemmatizer']=df['text_no_stopwords'].apply(lambda x: lemmatizing(x))
df.head()

,id,url_legal,license,text,y,standard_error,lang,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,en,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,..."
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,en,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey..."
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,en,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca..."
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,en,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round..."
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,en,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou..."


In [8]:
df['text_clean']=df['text_lemmatizer'].apply(lambda x: ' '.join(x))
df.head()

,id,url_legal,license,text,y,standard_error,lang,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,en,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,en,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,en,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,en,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,en,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...


In [9]:
df['sentence_len']=df['text'].apply(lambda x: len(x)-x.count(' '))
df['word_len']=df['text_tokenized'].apply(lambda x: len(x))
def count_punct(text):
    ct=sum([1 for char in text if char in string.punctuation])
    return round(ct/(len(text)-text.count(' ')),3)
df['punct_count']=df['text'].apply(lambda x:count_punct(x))
df.head()

,id,url_legal,license,text,y,standard_error,lang,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean,sentence_len,word_len,punct_count
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,en,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...,819,179,0.033
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,en,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...,774,169,0.072
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,en,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...,747,166,0.063
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,en,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...,747,164,0.044
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,en,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...,577,147,0.055


In [ ]:
# Named-Entity Recognition
import spacy
ner = spacy.load('en_core_web_lg')

# txt = df['text'].iloc[0]
# doc = ner(txt)
# spacy.displacy.render(doc,style='ent')

In [ ]:
## tag text and exctract tags into a list
df["tags"] = df["text"].apply(lambda x: [(tag.text, tag.label_) 
                                for tag in ner(x).ents] 
                             )
## utils function to count the element of a list
def utils_lst_count(lst):
    dic_counter = collections.Counter()
    for x in lst:
        dic_counter[x] += 1
    dic_counter = collections.OrderedDict( 
                     sorted(dic_counter.items(), 
                     key=lambda x: x[1], reverse=True))
    lst_count = [ {key:value} for key,value in dic_counter.items() ]
    return lst_count

df["tags"] = df["tags"].apply(lambda x: utils_lst_count(x))

## utils function create new column for each tag category
def utils_ner_features(lst_dics_tuples, tag):
    if len(lst_dics_tuples) > 0:
        tag_type = []
        for dic_tuples in lst_dics_tuples:
            for tuple in dic_tuples:
                type, n = tuple[1], dic_tuples[tuple]
                tag_type = tag_type + [type]*n
                dic_counter = collections.Counter()
                for x in tag_type:
                    dic_counter[x] += 1
        return dic_counter[tag]
    else:
        return 0

## extract features
tags_set = []
for lst in df["tags"].tolist():
     for dic in lst:
          for k in dic.keys():
              tags_set.append(k[1])
tags_set = list(set(tags_set))
for feature in tags_set:
     df["tags_"+feature] = df["tags"].apply(lambda x: 
                             utils_ner_features(x, feature))

df.head()

In [ ]:
# N-grams

In [10]:
# Word embedding

import gensim
import gensim.downloader
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts
# print(list(gensim.downloader.info()['models'].keys()))
# glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

# require data format for gensim
list_of_list = [row.split(' ') for row in df.text_clean]
print (len(list_of_list))
list_of_list[:2]


tagged_data = [TaggedDocument(words=doc,tags=[str(i)]) for i, doc in enumerate(list_of_list)]
tagged_data[:1]

d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)
d2v_model.build_vocab(tagged_data)
d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

2834


In [11]:
df['d2v_vector'] = df['text_lemmatizer'].apply(lambda x:d2v_model.infer_vector(x))

df.head()

,id,url_legal,license,text,y,standard_error,lang,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean,sentence_len,word_len,punct_count,d2v_vector
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,en,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...,819,179,0.033,"[-1.0434451, -0.44792616, 0.16821179, -0.87052..."
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,en,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...,774,169,0.072,"[-0.2723027, 0.505543, -0.3870601, -0.28024077..."
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,en,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...,747,166,0.063,"[-0.13357475, -0.7145384, 0.8714408, 0.8256378..."
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,en,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...,747,164,0.044,"[-1.0138851, 1.8109201, -1.0011533, -0.6449585..."
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,en,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...,577,147,0.055,"[-0.022582637, 0.22305238, 0.46563417, -0.8673..."


In [12]:
d2v_features = pd.DataFrame(df["d2v_vector"].to_list(), columns=['d2v_feature'+str(i) for i in range(40)])
df2 = df.drop(columns = ['id','url_legal','license','text','y','standard_error','lang',
#                          'tags',
 'text_no_punct','text_tokenized','text_no_stopwords','text_lemmatizer','text_clean','d2v_vector'
                        ])

train_final = df2.join(d2v_features)
labels = df["y"]

In [13]:
train_final.columns

Index(['sentence_len', 'word_len', 'punct_count', 'd2v_feature0',
       'd2v_feature1', 'd2v_feature2', 'd2v_feature3', 'd2v_feature4',
       'd2v_feature5', 'd2v_feature6', 'd2v_feature7', 'd2v_feature8',
       'd2v_feature9', 'd2v_feature10', 'd2v_feature11', 'd2v_feature12',
       'd2v_feature13', 'd2v_feature14', 'd2v_feature15', 'd2v_feature16',
       'd2v_feature17', 'd2v_feature18', 'd2v_feature19', 'd2v_feature20',
       'd2v_feature21', 'd2v_feature22', 'd2v_feature23', 'd2v_feature24',
       'd2v_feature25', 'd2v_feature26', 'd2v_feature27', 'd2v_feature28',
       'd2v_feature29', 'd2v_feature30', 'd2v_feature31', 'd2v_feature32',
       'd2v_feature33', 'd2v_feature34', 'd2v_feature35', 'd2v_feature36',
       'd2v_feature37', 'd2v_feature38', 'd2v_feature39'],
      dtype='object')

In [14]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score
# from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor()
scores = cross_val_score(rf,train_final,labels,cv=4,scoring='r2')
print('r2:')
print(scores)
scores_mse = cross_val_score(rf,train_final,labels,cv=4,scoring='neg_mean_squared_error')
print('mse:')
print(-1*scores_mse)

r2:
[0.19394467 0.40366017 0.48649138 0.32419479]
mse:
[0.66816157 0.66085521 0.70474257 0.61145168]


In [ ]:
from xgboost import XGBRegressor

### Train Final XGBoost
xg = XGBRegressor()
scores = cross_val_score(xg,train_final,labels,cv=4,scoring='r2')
print(scores)

In [ ]:
# word2vec
# from gensim.models import Word2Vec
# model = Word2Vec(sent,sg = 1)
# model.wv['young']
# model.wv.most_similar(positive='young')

In [ ]:
# Mean number of syllables per word,
# Mean number of words per sentence,
# Mean number of words considered “difficult” in a sentence (a word is “difficult” if it is not part of an “easy” words reference list),
# Part-of-Speech (POS) tags count per book,
# Readability formulas such as Flesch-Kincaid and
# Number of polysyllables (more than 3 syllables).
# rare words
# Name identification using NER to avoid incorrect lemmatization

In [ ]:
# LSTM

from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, LSTM
from keras.layers.embeddings import Embedding

In [ ]:
# tokenization
X_train, X_test, y_train, y_test = train_test_split(df['text_clean'],df['y'], test_size=0.30, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(len(X_train), len(X_train[0]))

In [ ]:
# padding
vocab_size = len(tokenizer.word_index) + 1

maxlen = 110

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
print(len(X_train), len(X_train[0]))

In [ ]:
# load glove wikipedia embedding
embeddings_dictionary = dict()
glove_file = open(r'D:/datasets/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix.shape

In [ ]:
# LSTM model
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
# model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, batch_size=256, epochs=20, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
print("Test Scores:", score)

In [25]:
### TfidfVectorizer

# from sklearn.feature_extraction.text import  TfidfVectorizer
# count_vect=TfidfVectorizer()
# x_counts=count_vect.fit_transform(raw_data['body_text'])
# print(x_counts.shape)
# print(count_vect.get_feature_names())


# def vectorize(df, tfidf, target='text_clean'):
#     tfidf_vect=tfidf.fit(df[target])
#     X_tfidf = tfidf_vect.transform(df[target])
    
#     words = tfidf_vect.get_feature_names()    
#     df_tfidf = pd.DataFrame(X_tfidf.toarray())
#     df_tfidf.columns = words
#     return(df_tfidf)

# tfidf = TfidfVectorizer()
# text_format=vectorize(df,tfidf)
# text_format.head()

# data=raw_data.merge(text_format, left_index=True, right_index=True)
# print(data.shape)
# data=data.set_index('id_x')
# data.head(2)
# data = pd.get_dummies(data, columns=['license_x'],prefix='license')

,000,002,003,004,006,01,015,03,05,074,...,µv,½d,ædui,ægidus,æmilius,æneas,æolian,æquians,æschylus,ça
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
text_format

,000,002,003,004,006,01,015,03,05,074,...,µv,½d,ædui,ægidus,æmilius,æneas,æolian,æquians,æschylus,ça
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from pycaret.regression import *
py_x = pd.concat([train_final, labels], axis=1)

exp_reg101 = setup(data = py_x, target = 'y', session_id=123)
best = compare_models(exclude = ['ransac'])

In [ ]:
from transformers import BertTokenizer, TFBertModel